In [ ]:
#!/usr/bin/python3
# coding: utf-8
# Hokkaido

In [ ]:
from datetime import datetime as dt
import sys
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
#import plotly.tools as tls
import plotly.graph_objects as go
#import plotly.io as pio
import plotly.offline as offline
import sys
if "ipy" in sys.argv[0]:
    offline.init_notebook_mode()
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly

In [ ]:
today_str = dt.now().isoformat()[:16].replace('T', ' ')
update_map = False

In [ ]:
with open("csv/maps.tmp", "rt") as f:
    prev = int(f.read())
imgname = "hokkaido-map.jpg"
tw_body = '北海道 新型コロナ 発生マップ (' + today_str + ')'
df=pd.read_csv("csv/maps.csv", encoding='shift-jis', header=0)

In [ ]:
if len(df) > prev:
    update_map = True
    with open("csv/maps.tmp", "wt") as f:
        f.write("{}".format(len(df)))

In [ ]:
#update_map = True

In [ ]:
fig = px.scatter_mapbox(df,
    title=tw_body,
    lat="lat", lon="lon", color="patients", size='indoor',
    color_continuous_scale=plotly.colors.sequential.Bluered,
    hover_name="name", 
    hover_data=["state", "category", "opened"],
    labels={'lat':'緯度', 'lon':'経度', 'state':'自治体',
            'patients':'感染者数', 'indoor':'室内',
            'category':'形態', 'name':'名称', 'id':'ID',
            'opened':'認定日'},
    size_max=12, zoom=7, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_layout(template='plotly_dark')
if update_map:
    tw = get_twitter()
    show_and_save_plotly(fig, imgname, js=False, show=True, image=True, html=True)
    tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido-map.html "
    tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)
else:
    print("nothin to tweet about hokkaido cluster map.")

In [ ]:
def hokkaido_cluster_pie(column_name, title, imgname, update_map,
                        labels01=None):
    states = df.groupby(column_name).size()
    if not labels01:
        labels = states.index.tolist()
    else:
        labels = []
        for i in states.index.tolist():
            labels.append(labels01[i])
    tw_body = '北海道 新型コロナ発生 {} ({})'.format(title, today_str)
    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=states.values.tolist(),
        textinfo='percent',
        title=tw_body
    )])
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    if update_map:
        show_and_save_plotly(fig, imgname, js=False)
        tw_body += " (夜の街関連を除く) "
        tw_body += " https://geneasyura.github.io/cov19-hm/hokkaido-map.html "
        tweet_with_image(tw, "docs/images/{}".format(imgname), tw_body)

In [ ]:
hokkaido_cluster_pie(
    'state', '市町村別', "hokkaido-cluster-state.jpg", update_map)

In [ ]:
hokkaido_cluster_pie(
    'category', '施設別', "hokkaido-cluster-category.jpg", update_map)

In [ ]:
hokkaido_cluster_pie(
    'indoor', '室内発生率(憶測)', "hokkaido-cluster-indoor.jpg",
    update_map, labels01=['室外', '室内'])

In [ ]:
hokkaido_cluster_pie(
    'voice', '会話発生率(憶測)', "hokkaido-cluster-voice.jpg",
    update_map, labels01=['会話なし', '会話あり'])

In [ ]:
hokkaido_cluster_pie(
    'loud', '大声発生率(憶測)', "hokkaido-cluster-loud.jpg",
    update_map, labels01=['大声なし', '大声あり'])

In [ ]:
hokkaido_cluster_pie(
    'H', '医療機関/福祉施設率(概算)', "hokkaido-cluster-hosp.jpg",
    update_map, labels01=['それ以外', '医療機関'])

In [ ]:
hokkaido_cluster_pie(
    'RoomType', '部屋種別(憶測)', "hokkaido-cluster-room.jpg",
    update_map)

In [ ]:
def df_to_html(filename):
    with open(filename, "w", encoding='utf-8') as f:
        f.write('<style>\n')
        f.write('.MapsTblColNo { width:  35px; }\n')
        f.write('.MapsTblColSt { width: 100px; }\n')
        f.write('.MapsTblColCt { width: 150px; }\n')
        f.write('.MapsTblColNm { width: 350px; }\n')
        f.write('.MapsTblColOp { width: 120px; }\n')
        f.write('.MapsTblColPt { width:  80px; }\n')
        f.write('.MapsTblColRt { width: 120px; }\n')
        f.write('</style>\n')

        f.write('<table border="0">\n')
        f.write('  <thead style="display: block;" bgcolor="#000000">\n')
        f.write('    <tr style="text-align: left;">\n')
        f.write('      <th class="MapsTblColNo">No.</th>\n')
        f.write('      <th class="MapsTblColSt">自治体</th>\n')
        f.write('      <th class="MapsTblColCt">施設種別</th>\n')
        f.write('      <th class="MapsTblColNm">名称</th>\n')
        f.write('      <th class="MapsTblColOp">認定日</th>\n')
        f.write('      <th class="MapsTblColPt">感染者数</th>\n')
        f.write('      <th class="MapsTblColRt">部屋種別</th>\n')
        f.write('    </tr>\n')
        f.write('  </thead>\n')
        f.write('  <tbody style="display: block; overflow-x: hidden; overflow-y: scroll; height: 300px;">\n')
        for i, r in df.iterrows():
            f.write('<tr bgcolor="{}">'.format(["#606060", "#000000"][i % 2]))
            f.write('<td class="MapsTblColNo">{}</td>'.format(r['no']))
            f.write('<td class="MapsTblColSt">{}</td>'.format(r['state']))
            f.write('<td class="MapsTblColCt">{}</td>'.format(r['category']))
            f.write('<td class="MapsTblColNm">{}</td>'.format(r['name']))
            f.write('<td class="MapsTblColOp">{}</td>'.format(r['opened']))
            f.write('<td class="MapsTblColPt">{}</td>'.format(r['patients']))
            f.write('<td class="MapsTblColRt">{}</td>'.format(r['RoomType']))
            f.write('</tr>\n')
        f.write('</tbody></table>\n')

In [ ]:
df_to_html('docs/_includes/maps.html')